In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from Napoleon import NapoleonClassifier

In [2]:
df = pd.read_csv('https://stepik.org/media/attachments/course/57984/titanic.csv', index_col=0)

In [3]:
bins = [-1]
bins = np.append(bins, df.describe()['Fare'][4:-1].values)
bins = np.append(bins, df['Fare'].max())
bins

array([ -1.    ,   7.9104,  14.4542,  31.    , 512.3292])

In [4]:
df['Category_fare'] = pd.cut(df['Fare'], bins=bins, labels=['Low', 'Mid', 'High_Mid', 'High'])

In [5]:
df.Embarked.fillna(df.Embarked.mode()[0], inplace = True)

In [6]:
df.Cabin = df.Cabin.fillna('NA')

In [7]:
df['Salutation'] = df.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
grp = df.groupby(['Sex', 'Pclass', 'Salutation'])
df.Age = grp.Age.apply(lambda x: x.fillna(x.median()))

In [8]:
Y = df['Survived']
X = df[df.columns[1:]]

In [9]:
X['Deck'] = X.Cabin.str[0]

In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X['Sex'] = le.fit_transform(X['Sex'])

In [11]:
X['Cabin'] = LabelEncoder().fit_transform(X['Cabin'])
X['Embarked'] = LabelEncoder().fit_transform(X['Embarked'])

In [12]:
pd.concat([X, pd.get_dummies(X.Category_fare, prefix="Fare")], axis=1).head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Category_fare,Salutation,Deck,Fare_Low,Fare_Mid,Fare_High_Mid,Fare_High
PassengerId,,,,,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,146,2,Low,Mr,N,1,0,0,0
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,81,0,High,Mrs,C,0,0,0,1
3,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,146,2,Mid,Miss,N,0,1,0,0
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,55,2,High,Mrs,C,0,0,0,1
5,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,146,2,Mid,Mr,N,0,1,0,0


In [13]:
cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']



In [14]:
# Разделяем выборку на обучающую и тестовую

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X[cols], Y, test_size=0.3, random_state=255)

print(X_train.shape, X_test.shape)

(623, 8) (268, 8)


In [15]:
X[cols].values

array([[  3.    ,   1.    ,  22.    , ...,   7.25  , 146.    ,   2.    ],
       [  1.    ,   0.    ,  38.    , ...,  71.2833,  81.    ,   0.    ],
       [  3.    ,   0.    ,  26.    , ...,   7.925 , 146.    ,   2.    ],
       ...,
       [  3.    ,   0.    ,  18.    , ...,  23.45  , 146.    ,   2.    ],
       [  1.    ,   1.    ,  26.    , ...,  30.    ,  60.    ,   0.    ],
       [  3.    ,   1.    ,  32.    , ...,   7.75  , 146.    ,   1.    ]])

##  Model

In [16]:
import itertools
for i,b in enumerate(itertools.product([10,20],[2,4])):
    my_other_object =NapoleonClassifier(max_depth=b[0],min_samples=b[1],criterion='entropy')
    tree = my_other_object.fit(X_train, Y_train)
    pr = my_other_object.prediction(X_train)
    
    err_tr = roc_auc_score(Y_train,pr)
    pr_t = my_other_object.prediction(X_test)
    err_te = roc_auc_score(Y_test,pr_t)
    print('max_depth:',b[0],'min_samples:',b[1], 'Error train:',np.round(err_tr,2),'Error test:',np.round(err_te,2))

max_depth: 10 min_samples: 2 Error train: 0.89 Error test: 0.76
max_depth: 10 min_samples: 4 Error train: 0.87 Error test: 0.76
max_depth: 20 min_samples: 2 Error train: 0.89 Error test: 0.75
max_depth: 20 min_samples: 4 Error train: 0.87 Error test: 0.75
